<a href="https://colab.research.google.com/github/kordusmonika/dw_matrix_car/blob/master/day4_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [49]:
cd /content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_2/dw_matrix_car/data

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_2/dw_matrix_car/data


In [50]:
df = pd.read_hdf('car.h5')
df.shape

(106494, 155)

#Feature Engineering

In [0]:
SUFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[ feat + SUFIX_CAT ] = factorized_values

In [52]:
cat_feats = [x for x in df.columns if SURFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

#Decision Tree

In [54]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015937)

In [55]:
run_model(RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0), cat_feats)

(-18718.657185256638, 64.5424578125788)

#XGBRegressor

In [56]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats )

[20:11:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

#Permutation Importance for XGBoost

In [57]:
m = xgb.XGBRegressor(**xgb_params)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[20:13:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1274 ± 0.0017,param_napęd__cat
0.1159 ± 0.0025,param_stan__cat
0.1131 ± 0.0037,param_rok-produkcji__cat
0.0565 ± 0.0010,param_faktura-vat__cat
0.0482 ± 0.0014,param_moc__cat
0.0416 ± 0.0017,param_skrzynia-biegów__cat
0.0252 ± 0.0010,param_marka-pojazdu__cat
0.0247 ± 0.0014,feature_kamera-cofania__cat
0.0232 ± 0.0008,param_typ__cat
0.0198 ± 0.0006,param_pojemność-skokowa__cat


In [0]:
feats = ['param_napęd__cat',
'param_stan__cat',
'param_rok-produkcji__cat',
'param_faktura-vat__cat',
'param_moc__cat',
'param_skrzynia-biegów__cat',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'param_wersja__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'feature_system-start-stop__cat',
'param_kod-silnika__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_światła-led__cat', 
'feature_czujniki-parkowania-przednie__cat']

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x) )

In [61]:
feats = ['param_napęd__cat',
'param_stan__cat',
'param_faktura-vat__cat',
'param_rok-produkcji',
'param_moc__cat',
'param_skrzynia-biegów__cat',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'param_wersja__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'feature_system-start-stop__cat',
'param_kod-silnika__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_światła-led__cat', 
'feature_czujniki-parkowania-przednie__cat']


run_model(xgb.XGBRegressor(**xgb_params), feats )

[20:19:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:19:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:19:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11386.828140541147, 58.212963494024464)

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]) )

In [69]:
feats = ['param_napęd__cat',
'param_stan__cat',
'param_faktura-vat__cat',
'param_rok-produkcji',
'param_moc',
'param_skrzynia-biegów__cat',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'param_wersja__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'feature_system-start-stop__cat',
'param_kod-silnika__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_światła-led__cat', 
'feature_czujniki-parkowania-przednie__cat']


run_model(xgb.XGBRegressor(**xgb_params), feats )

[20:24:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9726.018375471673, 80.87269502689219)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ', '') ) )

In [73]:
feats = ['param_napęd__cat',
'param_stan__cat',
'param_faktura-vat__cat',
'param_rok-produkcji',
'param_moc',
'param_skrzynia-biegów__cat',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa',
'seller_name__cat',
'param_wersja__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'feature_system-start-stop__cat',
'param_kod-silnika__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_światła-led__cat', 
'feature_czujniki-parkowania-przednie__cat']


run_model(xgb.XGBRegressor(**xgb_params), feats )

[20:30:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:30:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:30:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9556.21383364917, 101.81252798368146)